In [10]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [3]:
class planet():
    "a planet in our solar system"
    def __init__(self, semimajor, eccentricity):
        self.x=np.zeros(2) #x and y position
        self.v=np.zeros(2) #x and y velocity
        self.a_g=np.zeros(2) #x and y accelaration
        self.t=0.0 #current time
        self.dt=0.0 #current timestep
        self.a=semimajor #semimajor axis of the orbit
        self.e=eccentricity #eccentricity integer timestep1
        self.istep=0 #current integer timestep1
        self.name="" #name for the planet

## Define a dictionary with some constants

In [5]:
solar_system={"M_sun":1.0, "G":39.4784176043574320}

## Define some functions for setting circular velocity and acceleration

In [12]:
def SolarCircularVelocity(p):
    G= solar_system["G"]
    M=solar_system["M_sun"]
    r=(p.x[0]**2+p.x[1]**2)**.5
    
    #return the circulat velocity
    return (G*M/r)**.5

In [13]:
def SolarGravitationalAcceleration(p):
    
    G=solar_system["G"]
    M=solar_system["M_sun"]
    r=(p.x[0]**2+p.x[1]**2)**.5
    
    #accelaration in AU/yr/yr
    a_grav=-1.0*G*M/r**2
    
    #find the angle at this position
    if(p.x[0]==0.0):
        if(p.x[1]>0.0):
            theta=.5*np.pi
        else:
            theta=1.5*np.pi
    else:theta=np.arctan2(p.x[1],p.x[0])
        
    #set the x and y components of the velocity
    #p.a_g[0]=a_grav*np.cos(theta)
    #p.a_g[1]=a_Grav*np.sin(theta)
    return a_grav*np.cos(theta), a_grav*np.sin(theta)


## Compute timestep

In [7]:
def calc_dt(p):
    #integration tolerance
    ETA_TIME_STEP=.0004
    
    #compute the timestep
    eta=ETA_TIME_STEP
    v= (p.v[0]**2+p.v[1]**2)**.5
    a=(p.a_g[0]**2+p.a_g[1]**2)**.5
    dt=eta*np.fmin(1.0/np.fabs(v),1.0/np.fabs(a)**.5)
    
    return dt

In [28]:
def SetPlanet(p,i):
    AU_in_km=1.495979e+8
    #circular velocity
    v_c=0.0     #circular velocity in AU/yr
    v_e=0.0     #velocity at perihelion in AU/yr
    
    #planet by planet initial conditions
    
    #Mercury
    if(i==0):
        #semi major axis in AU
        p.a=57909227.0/AU_in_km
        
        #eccentricity
        p.e=.20563593
        
        #name
        p.name="Mercury"
    #venus
    elif(i==1):
        #semi-major axis in AU
        p.a=108209475.0/AU_in_km
        
        #eccentricity
        p.e=.00677672
        p.name="Venus"
    elif(i==2):
        p.a=1.0
        #eccentricity
        p.e=.01671123
        p.name="Earth"
        
    #set remaining properties
    p.t=0.0
    p.x[0]=p.a*(1.0-p.e)
    p.x[1]=0.0
    
    #get equiv circular velocity
    v_c=SolarCircularVelocity(p)
    
    #velocity at perihelion
    v_c=v_c*(1+p.e)**.5
    
    #set velocity
    p.v[0]=0.0 #no x velocity at perihelion
    p.v[1]=v_e #y velocity at perihelion (counterclockwise)
    
    #calculate gravitational accelaration form sun
    p.a_g=SolarGravitationalAcceleration(p)
    
    #set timestep
    p.dt=calc_dt(p)

## Write leapfrog integrator

In [29]:
def x_first_step(x_i, v_i, a_i, dt):
    #x_1/2=x_0+1/2 v_0 Delta_t +1/4 a_0 Delta t^2
    return x_i +.5*v_i*dt+.25*a_i*dt**2

In [30]:
def v_full_step(v_i, a_ipoh, dt):
    #v_i+1=v_i+a_i+1/2 Delta t
    return v_i+a_ipoh*dt

In [31]:
def x_full_step(x_ipoh, v_ipl, a_ipoh, dt):
    #x_3/2=x_1/2+v_i+1 Delta t
    return x_ipoh+v_ipl*dt

## Write a function to save the data fil

In [44]:
def SaveSolarSystem(p, n_planets, t, dt, istep, ndim):
    #loops over the number of planets
    for i in range(n_planets):
        #define a filename
        fname="planet.%s.text" %p[i].name
        
        if (istep==0):
            #create the file on the first timestep
            fp=open(fname,"w")
        else:
            #append the file on the subsequent timesteps
            fp=open(fname,"a")
        
        #compute the drifted properties of the planet
        v_drift=np.zeros(ndim)
        
        for k in range(ndim):
            v_drift[k]=p[i].v[k]+.5*p[i].a_g[k]*p[i].dt
            
        #write the data to file
        s="%6d\t%6.5f\t%6.5f\t%6d\t%6.5f\t% 6.5f\t% 6.5f\t% 6.5f\t% 6.5f\t% 6.5f\t% 6.5f\t% 6.5f\n%" \
            % (istep,t,dt,p[i].istep,p[i].t,p[i].dt,p[i].x[0],p[i].x[1],v_drift[0],v_drift[1], \
               p[i].a_g[0],p[i].a_g[1])
        fp.write(s)
                                           
        #close the file
        fp.close()

## Write a function to evolve the solar system

In [45]:
def EvolveSolarSystem(p,n_planets,t_max):
    
    #number of spatial dimensions
    ndim=2
    
    #define the first timestep
    dt=.5/365.25
    
    #define the starting time
    t=0.0
    
    #define the starting timestep
    istep=0
    
    #save the initial conditions
    SaveSolarSystem(p,n_planets,t,dt,istep,ndim)
    
    #begin a loop over the global timescale
    while(t<t_max):
        #check to see if the next step exceeds to max time. If so, smaller step
        if(t+dt>t_max):
            dt=t_max-t   #limit the step to align with t_max
            
            #evolve each planet
            for i in range(n_planets):
                while (p[i].istep==0):
                    
                    #take the first step according to a verlet scheme
                    for k in range(ndim):
                        p[i].x[k]=x_first_step(p[i].x[k],p[i].v[k],p[i].a_g[k],p[i].dt)
                        
                    #update the acceleration
                    p[i].a_g=SolarGravitationalAcceleration(p[i])
                    
                    #update the time by 1/2dt
                    p[i].t+=.5*p[i].dt
                    
                    #update the timestep
                    p[i].dt=colc_dt(p[i])
                    
                #continue with a normal step
                
                #limit to align with the global timestep
                if (p[i].t +p[i].dt>t+dt):
                    p[i].dt=t+dt-p[i].t
                    
                #evolve the velocity
                for k in range(ndim):
                    p[i].c[k]=v_full_step(p[i].v[k],p[i].a_g[k],p[i].dt)
                    
                #evolve position
                for k in range(ndim):
                    p[i].x[k]=f_full_step(p[i].x[k],p[i].a_g[k],p[i].dt)
                    
                #update the acceleration
                p[i].a_g=SolarGravitationalAcceleration(p[i])
                
                #update by dt
                p[i].t+=p[i].dt
                
                #compute the new timestep
                p[i].t+=p[i].dt
                
                #compute the enw timestep
                p[i].dt=calc_dt(p[i])
                
                #update the planet's timestep
                p[i].istep+=1
                
            #now update the global system time
            t+=dt
            
            #update the global system time
            t+=dt
            
            #update the global step number
            istep+=1
            
            #output the current state
            SaveSolarSystem(p,n_planets,t,dt,istep,ndim)
        #print the final steps and time
        print("Time t=",t)
        print("Maximum t =", t_max)
        print("Maximum number of steps =", istep)
        #end of evolution

## Create a routine to read the data

In [46]:
def read_twelve_arrays(fname):
    fp=open(fname,'r')
    fl=fp.readlines()
    n=len(fl)
    a=np.zeros(n)
    b=np.zeros(n)
    c=np.zeros(n)
    d=np.zeros(n)
    f=np.zeros(n)
    g=np.zeros(n)
    h=np.zeros(n)
    j=np.zeros(n)
    k=np.zeros(n)
    l=np.zeros(n)
    m=np.zeros(n)
    p=np.zeros(n)
    for i in range(n):
        a[i]=float(fl[i].split()[0])
        b[i]=float(fl[i].split()[1])
        c[i]=float(fl[i].split()[2])
        d[i]=float(fl[i].split()[3])
        f[i]=float(fl[i].split()[4])
        g[i]=float(fl[i].split()[5])
        h[i]=float(fl[i].split()[6])
        j[i]=float(fl[i].split()[7])
        k[i]=float(fl[i].split()[8])
        l[i]=float(fl[i].split()[9])
        m[i]=float(fl[i].split()[10])
        p[i]=float(fl[i].split()[11])
    return a,b,c,d,f,g,h,j,k,l,m,n,p

## Perform the integration of the solar system

In [47]:
#set the number of planets
n_planets=3

#set the maximum time of the simulation
t_max=2.0

#create empty list of planets
p=[]

#set the planets
for i in range(n_planets):
    
    #create an empty planet
    ptmp=planet(0.0,0.0)
    
    #set the planet properties
    SetPlanet(ptmp,i)
    #remember the planet
    p.append(ptmp)
    
#evolve the solar system
EvolveSolarSystem(p,n_planets,t_max)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in double_scalars
  if __name__ == '__main__':


ValueError: incomplete format